In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
import requests
import json
from pandas.io.json import json_normalize
from itertools import islice

In [0]:
# Envelope rond de provinciegrenzen doorzoeken. Alle objectids die daar binnen vallen downloaden en op een rijtje zetten.
envelope = '93003.99999999802+464251.5000000022+154328.6700000043+580999.9999999997' # Polygoon van NH trekt hij niet.
# In deze query wordt wegdeel_v gedownload. Dat is laag nummer 112. Aanpassen voor andere dataset.
laag = 'https://geoservices.noord-holland.nl/ags/rest/services/oi_op/oi_areaal/MapServer/97'

In [0]:
laaginfo = requests.get(laag+ '?f=pjson')
info = json_normalize(json.loads(laaginfo.content))
info_df = json_normalize(info.fields[0])
fieldsdf = info_df.name
fields = fieldsdf.tolist()
fields

In [0]:
query = laag +'/query?where=&text=&objectIds=&time=&geometry='+ envelope + '&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=true&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
r =  requests.get(query, verify=False)
df = json_normalize(json.loads(r.content))
df.objectIds[0][0]
objectIds_list = df.objectIds[0]

In [0]:
# Velden kiezen die je wilt downloaden en in het juiste format zetten of alles houden.
#fields = ['SHAPE','WEGBEHSRT','WEGNUMMER','WEGDEELLTR','HECTO_LTTR','BAANSUBSRT','RPE_CODE', 'ADMRICHTNG', 'RIJRICHTNG','STT_NAAM','WPSNAAMNEN','OBJECTID_1','OBJECTID']
fieldsstring = ''
for f in range(0,len(fields)):
    field = fields[f]
    if fieldsstring == '':
        fieldsstring = field
    else:
        fieldsstring = fieldsstring + '%2C+' + field

In [0]:
# objectIds opsplitsen in chunks
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())
n = 100 # grootte van chunks
objectIds_chunks = list(chunk(objectIds_list, n))
aantalchunks = len(objectIds_chunks)

In [0]:
# Areaaldata downloaden per chunk
#df_PNH = pd.DataFrame()
for i in range(0,aantalchunks):
    chunk = objectIds_chunks[i]
    objectids = ''
    for j in range(0,len(chunk)):
        objectid = str(chunk[j])
        if objectids == '':
            objectids = objectid
        else:
            objectids = objectids + '%2C+' + objectid
    query2=laag+'/query?where=&text=&objectIds='+ objectids +'&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields='+ fieldsstring + '&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
    r =  requests.get(query2, verify=False)
    df2 = json_normalize(json.loads(r.content))
    data = json_normalize(df2.features[0])
    data['ID'] = 97
    df_PNH = df_PNH.append(data)

In [0]:
df_PNH.tail()

In [0]:
df_object = df.append(df_PNH)

In [0]:
len(df_object)

In [0]:
len(df)

In [0]:
df_object.head()

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("All_road_objects")
df = mydataset.get_dataframe()

In [0]:
# Recipe outputs
final_road_objects = dataiku.Dataset("final_road_objects")
final_road_objects.write_with_schema(df_object)